In [1]:
import re
import nltk
!pip install contractions
import contractions

In [2]:
import pandas as pd
data = pd.read_csv("compiled.csv")

In [3]:
data.head()

,Unnamed: 0,label,target,is_it_hate
0,0,u really think i would not have been raped by ...,True,1
1,1,the uk has threatened to return radioactive wa...,True,1
2,2,if english is not imposition then hindi is als...,True,1
3,3,no liberal congratulated hindu refugees post c...,True,1
4,4,he said bro even your texts sound redneck what...,True,1


In [4]:
data.drop('Unnamed: 0', axis=1, inplace=True)

In [5]:
data.label[1254]

'smoking fiya in a hotel eating shrimp wit a towel under the door and a thick red hoe'

In [6]:
data["label"] = data["label"].apply(lambda x: x.lower())


In [7]:
%time
data["label"] = data["label"].apply(lambda x: contractions.fix(x))



CPU times: user 0 ns, sys: 1e+03 ns, total: 1e+03 ns
Wall time: 2.62 µs


In [8]:
data.label[1254]

'smoking fiya in a hotel eating shrimp wit a towel under the door and a thick red hoe'

In [9]:
def remove_URL(text):
    return re.sub(r"https?://\S+|www\.\S+", "", text)


data["label"] = data["label"].apply(lambda x: remove_URL(x))

In [10]:
def remove_html(text):
    html = re.compile(r"<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});")
    return re.sub(html, "", text)

data["label"] = data["label"].apply(lambda x: remove_html(x))

In [11]:
def remove_non_ascii(text):
    """
        Remove non-ASCII characters 
    """
    return re.sub(r'[^\x00-\x7f]',r'', text) # or ''.join([x for x in text if x in string.printable]) 

data["label"] = data["label"].apply(lambda x: remove_non_ascii(x))

In [12]:
def remove_special_characters(text):
    """
        Remove special special characters, including symbols, emojis, and other graphic characters
    """
    emoji_pattern = re.compile(
        '['
        u'\U0001F600-\U0001F64F'  # emoticons
        u'\U0001F300-\U0001F5FF'  # symbols & pictographs
        u'\U0001F680-\U0001F6FF'  # transport & map symbols
        u'\U0001F1E0-\U0001F1FF'  # flags (iOS)
        u'\U00002702-\U000027B0'
        u'\U000024C2-\U0001F251'
        ']+',
        flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

data["label"] = data["label"].apply(lambda x: remove_special_characters(x))


In [13]:
def remove_punct(text):
    """
        Remove the punctuation
    """
    return re.sub(r'[]!"$%&\'()*+,./:;=#@?[\\^_`{|}~-]+', " ", text)
    #return text.translate(str.maketrans('', '', string.punctuation))

data["label"] = data["label"].apply(lambda x: remove_punct(x))

In [14]:
nltk.download("stopwords")
from nltk.corpus import stopwords

stop = set(stopwords.words('english'))
stop.add("rt")


data["label"] = data["label"].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/dipta007/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
data

,label,target,is_it_hate
0,really think would raped feral hindu muslim ba...,True,1
1,uk threatened return radioactive waste eu agre...,True,1
2,english imposition hindi also imposition shut ...,True,1
3,liberal congratulated hindu refugees post cab ...,True,1
4,said bro even texts sound redneck whatever fuc...,True,1
...,...,...,...
86913,stoning cut hands beheading tyre neck brace tr...,False,0
86914,better,False,0
86915,house famous lots homemade stuff including cla...,False,0
86916,could norwegian poster give us non norwegains ...,False,0


In [16]:
#remove tweets with just one word
data = data[data.label.str.contains(' ')]
data

,label,target,is_it_hate
0,really think would raped feral hindu muslim ba...,True,1
1,uk threatened return radioactive waste eu agre...,True,1
2,english imposition hindi also imposition shut ...,True,1
3,liberal congratulated hindu refugees post cab ...,True,1
4,said bro even texts sound redneck whatever fuc...,True,1
...,...,...,...
86912,simply copy paste following text youtube video...,False,0
86913,stoning cut hands beheading tyre neck brace tr...,False,0
86915,house famous lots homemade stuff including cla...,False,0
86916,could norwegian poster give us non norwegains ...,False,0


In [17]:
#remove tweets with just one word
data = data[data.label.str.contains(' ')]
data

,label,target,is_it_hate
0,really think would raped feral hindu muslim ba...,True,1
1,uk threatened return radioactive waste eu agre...,True,1
2,english imposition hindi also imposition shut ...,True,1
3,liberal congratulated hindu refugees post cab ...,True,1
4,said bro even texts sound redneck whatever fuc...,True,1
...,...,...,...
86912,simply copy paste following text youtube video...,False,0
86913,stoning cut hands beheading tyre neck brace tr...,False,0
86915,house famous lots homemade stuff including cla...,False,0
86916,could norwegian poster give us non norwegains ...,False,0


In [18]:
data.dropna()

,label,target,is_it_hate
0,really think would raped feral hindu muslim ba...,True,1
1,uk threatened return radioactive waste eu agre...,True,1
2,english imposition hindi also imposition shut ...,True,1
3,liberal congratulated hindu refugees post cab ...,True,1
4,said bro even texts sound redneck whatever fuc...,True,1
...,...,...,...
86912,simply copy paste following text youtube video...,False,0
86913,stoning cut hands beheading tyre neck brace tr...,False,0
86915,house famous lots homemade stuff including cla...,False,0
86916,could norwegian poster give us non norwegains ...,False,0


In [19]:
data.to_csv("refined_hate.csv")

In [20]:
# from textblob import TextBlob

# from tqdm import tqdm
# tqdm.pandas()

# def correct_spell(text):
   
#     return TextBlob(text).correct()
#     #return text.translate(str.maketrans('', '', string.punctuation))

# da = data["label"].head().progress_apply(lambda x: ''.join(correct_spell(x)))
